In [ ]:
## EVALUATION METRICS
RMSE
plotta grafico con actual vs predicted e riga bisettrice
sulla x e la y i prezzi attuali e predetti

In [ ]:
Dataset split to test the performances of the model:

- 80% training
- 20% test to perform an evaluation that simulated realworld scenarios